<a href="https://colab.research.google.com/github/staufferkn/Course_Python_for_Data_Analysis/blob/main/Data_Wrangling_Join_and_Reshape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Chapter 8
#Data Wrangling: Join, Combine, and Reshape

In [ ]:
#In many applications, data may be spread across a number of files or databases, or be
#arranged in a form that is not convenient to analyze. This chapter focuses on tools to
#help combine, join, and rearrange data.

In [1]:
import pandas as pd
import numpy as np

In [ ]:
#8.1 Hierarchical indexing

#Hierarchical indexing is an important feature of pandas that enables you to have
#multiple (two or more) index levels on an axis. Another way of thinking about it
#is that it provides a way for you to work with higher dimensional data in a lower
#dimensional form. Let’s start with a simple example: create a Series with a list of lists
#(or arrays) as the index:

In [ ]:
data = pd.Series(np.random.uniform(size=9),
 ....: index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
 ....: [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [ ]:
data

a  1    0.178831
   2    0.050787
   3    0.091156
b  1    0.557763
   3    0.750401
c  1    0.772399
   2    0.160975
d  2    0.498082
   3    0.034090
dtype: float64

In [ ]:
#What you’re seeing is a prettified view of a Series with a MultiIndex as its index. The “gaps” in the index display mean “use the label directly above”:

In [ ]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [ ]:
#With a hierarchically indexed object, so-called partial indexing is possible, enabling
#you to concisely select subsets of the data:

In [ ]:
data["b"]

,0
1,0.446283
3,0.654270


In [ ]:
data["b":"c"]

b  1    0.446283
   3    0.654270
c  1    0.467824
   2    0.194644
dtype: float64

In [ ]:
data.loc[["b", "d"]]

b  1    0.446283
   3    0.654270
d  2    0.671136
   3    0.333949
dtype: float64

In [ ]:
#Selection is even possible from an “inner” level. Here I select all of the values having
#the value 2 from the second index level:

In [ ]:
data.loc[:, 2]

,0
a,0.902832
c,0.194644
d,0.671136


In [ ]:
#Hierarchical indexing plays an important role in reshaping data and in group-based
#operations like forming a pivot table. For example, you can rearrange this data into a
#DataFrame using its unstack method:

In [ ]:
data.unstack()

,1,2,3
a,0.639889,0.902832,0.032098
b,0.446283,NaN,0.654270
c,0.467824,0.194644,NaN
d,NaN,0.671136,0.333949


In [ ]:
#The inverse operation on unstack is stack

In [ ]:
data.unstack().stack()

a  1    0.639889
   2    0.902832
   3    0.032098
b  1    0.446283
   3    0.654270
c  1    0.467824
   2    0.194644
d  2    0.671136
   3    0.333949
dtype: float64

In [ ]:
#With a DataFrame, either axis can have a hierarchical index:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
....: index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
columns=[["Ohio", "Ohio", "Colorado"],
....: ["Green", "Red", "Green"]])

In [ ]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [ ]:
#The hierarchical levels can have names (as strings or any Python objects). If so, these
#will show up in the console output:

In [ ]:
frame.index.names = ["key1", "key2"]

In [ ]:
frame.columns.names = ["state", "color"]

In [ ]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
#These names supersede the name attribute, which is used only with single-level indexes
#Be careful to note that the index names "state" and "color" are
#not part of the row labels (the frame.index values).

In [ ]:
#You can see how many levels an index has by accessing its nlevels attribute:

In [ ]:
frame.index.nlevels

2

In [ ]:
#With partial column indexing you can similarly select groups of columns:

In [ ]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [ ]:
#A MultiIndex can be created by itself and then reused; the columns in the preceding
#DataFrame with level names could also be created like this:

In [ ]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
 ["Green", "Red", "Green"]],
 names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [ ]:
#Reordering and Sorting Levels

#At times you may need to rearrange the order of the levels on an axis or sort the data
#by the values in one specific level. The swaplevel method takes two level numbers
#or names and returns a new object with the levels interchanged (but the data is
#otherwise unaltered):

In [ ]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [ ]:
#sort_index by default sorts the data lexicographically using all the index levels, but
#you can choose to use only a single level or a subset of levels to sort by passing the
#level argument. For example:

In [ ]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [ ]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [ ]:
#Summary Statistics by Level

#Many descriptive and summary statistics on DataFrame and Series have a level
#option in which you can specify the level you want to aggregate by on a particular
#axis. Consider the above DataFrame; we can aggregate by level on either the rows or
#columns, like so:

In [ ]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [ ]:
frame.groupby(level="color", axis="columns").sum()

/tmp/ipython-input-775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [ ]:
#Indexing with a DataFrame's Columns

#It’s not unusual to want to use one or more columns from a DataFrame as the
#row index; alternatively, you may wish to move the row index into the DataFrame’s
#columns. Here’s an example DataFrame:

In [ ]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
 ....: "c": ["one", "one", "one", "two", "two",
 ....: "two", "two"],
 ....: "d": [0, 1, 2, 0, 1, 2, 3]})

In [ ]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [ ]:
#DataFrame’s set_index function will create a new DataFrame using one or more of
#its columns as the index:

In [ ]:
frame2 = frame.set_index(["c", "d"])

In [ ]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [ ]:
#By default, the columns are removed from the DataFrame, though you can leave
#them in by passing drop=False to set_index:

In [ ]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [ ]:
#reset_index, on the other hand, does the opposite of set_index; the hierarchical
#index levels are moved into the columns:

In [ ]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [ ]:
#8.2 Combining and Merging Datasets

In [ ]:
#Database-Style DataFrame Joins

In [ ]:
#Merge or join operations combine datasets by linking rows using one or more keys.
#These operations are particularly important in relational databases (e.g., SQL-based).
#The pandas.merge function in pandas is the main entry point for using these algorithms on your data.
#Let’s start with a simple example:

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
 ....: "data1": pd.Series(range(7), dtype="Int64")})

In [ ]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
 ....: "data2": pd.Series(range(3), dtype="Int64")})

In [ ]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [ ]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [ ]:
#This is an example of a many-to-one join; the data in df1 has multiple rows labeled a and b, whereas df2 has only one row for each value in the key column. Calling
#pandas.merge with these objects, we obtain:

In [ ]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [ ]:
#Note that I didn’t specify which column to join on. If that information is not
#specified, pandas.merge uses the overlapping column names as the keys. It’s a good
#practice to specify explicitly, though

In [ ]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [ ]:
#In general, the order of column output in pandas.merge operations is unspecified.

#If the column names are different in each object, you can specify them separately:

In [ ]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
 ....: "data1": pd.Series(range(7), dtype="Int64")})

In [ ]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
 ....: "data2": pd.Series(range(3), dtype="Int64")})

In [ ]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [ ]:
#You may notice that the "c" and "d" values and associated data are missing from
#the result. By default, pandas.merge does an "inner" join; the keys in the result are
#the intersection, or the common set found in both tables. Other possible options are
#"left", "right", and "outer". The outer join takes the union of the keys, combining
#the effect of applying both left and right joins:

In [ ]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [ ]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [ ]:
#In an outer join, rows from the left or right DataFrame objects that do not match
#on keys in the other DataFrame will appear with NA values in the other DataFrame’s
#columns for the nonmatching rows.

In [ ]:
#Many-to-many merges form the Cartesian product of the matching keys. Here’s an example

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
 ....: "data1": pd.Series(range(6), dtype="Int64")})

In [ ]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
"data2": pd.Series(range(5), dtype="Int64")})

In [ ]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [ ]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [ ]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [ ]:
#Since there were three "b" rows in the left DataFrame and two in the right one, there are six "b" rows in the result. The join method passed to the how keyword argument
#affects only the distinct key values appearing in the result:

In [ ]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [ ]:
#To merge with multiple keys, pass a list of column names:

In [ ]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
"key2": ["one", "two", "one"],
"lval": pd.Series([1, 2, 3], dtype="int64")})

In [ ]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
"key2": ["one", "one", "one", "two"],
"rval": pd.Series([4, 5, 6, 7], dtype="int64")})

In [ ]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3.0,6.0
1,bar,two,NaN,7.0
2,foo,one,1.0,4.0
3,foo,one,1.0,5.0
4,foo,two,2.0,NaN


In [ ]:
#To determine which key combinations will appear in the result depending on the
#choice of merge method, think of the multiple keys as forming an array of tuples to
#be used as a single join key.

In [ ]:
#A last issue to consider in merge operations is the treatment of overlapping column names. For example:

In [ ]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [ ]:
#pandas.merge has a suffixes option
#for specifying strings to append to overlapping names in the left and right DataFrame objects:

In [ ]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [ ]:
#Merging on Index

#In some cases, the merge key(s) in a DataFrame will be found in its index (row labels).
#In this case, you can pass left_index=True or right_index=True (or both) to
#indicate that the index should be used as the merge key:

In [ ]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
 ....: "value": pd.Series(range(6), dtype="Int64")})

In [ ]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [ ]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [ ]:
right1

,group_val
a,3.5
b,7.0


In [ ]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [ ]:
#If you look carefully here, you will see that the index values for
#left1 have been preserved, whereas in other examples above, the
#indexes of the input DataFrame objects are dropped. Because the
#index of right1 is unique, this “many-to-one” merge (with the
#default how="inner" method) can preserve the index values from
#left1 that correspond to rows in the output

In [ ]:
#Since the default merge method is to intersect the join keys, you can instead form the
#union of them with an outer join:

In [ ]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [ ]:
#With hierarchically indexed data, things are more complicated, as joining on index is
#equivalent to a multiple-key merge:

In [ ]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
 ....: "Nevada", "Nevada"],
 ....: "key2": [2000, 2001, 2002, 2001, 2002],
 ....: "data": pd.Series(range(5), dtype="Int64")})

In [ ]:
righth_index = pd.MultiIndex.from_arrays(
 ....: [
 ....: ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
 ....: [2001, 2000, 2000, 2000, 2001, 2002]
 ....: ]
 ....: )

In [ ]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="int64", index=righth_index),
"event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="int64", index=righth_index)})

In [ ]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [ ]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [ ]:
#In this case, you have to indicate multiple columns to merge on as a list (note the
#handling of duplicate index values with how="outer"):

In [ ]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [ ]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2.0,3.0
3,Nevada,2001,3,0.0,1.0
4,Nevada,2002,4,NaN,NaN
0,Ohio,2000,0,4.0,5.0
0,Ohio,2000,0,6.0,7.0
1,Ohio,2001,1,8.0,9.0
2,Ohio,2002,2,10.0,11.0


In [ ]:
#Using the indexes on both sides to merge is also possible

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
 ....: index=["a", "c", "e"],
 ....: columns=["Ohio", "Nevada"]).astype("Int64")

In [ ]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
 ....: index=["b", "c", "d", "e"],
 ....: columns=["Missouri", "Alabama"]).astype("Int64")

In [ ]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [ ]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [ ]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [ ]:
#DataFrame has a join instance method to simplify merging by index. It can also be
#used to combine many DataFrame objects having the same or similar indexes but
#nonoverlapping columns. In the prior example, we could have written:

In [ ]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [ ]:
#Compared with pandas.merge, DataFrame’s join method performs a left join on the
#join keys by default. It also supports joining the index of the passed DataFrame on
#one of the columns of the calling DataFrame:

In [ ]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [ ]:
#Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join
#as an alternative to using the more general pandas.concat function described in the next section:

In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
 ....: index=["a", "c", "e", "f"],
 ....: columns=["New York", "Oregon"])

In [ ]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [ ]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [ ]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


In [ ]:
#Concatenating Along an Axis
#Another kind of data combination operation is referred to interchangeably as concatenation or stacking. NumPy’s concatenate function can do this with NumPy arrays:

In [ ]:
arr = np.arange(12).reshape((3, 4))

In [ ]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [ ]:
#In the context of pandas objects such as Series and DataFrame, having labeled axes
#enable you to further generalize array concatenation. In particular, you have a number of additional concerns:

 #• If the objects are indexed differently on the other axes, should we combine the distinct elements in these axes or use only the values in common?
 #• Do the concatenated chunks of data need to be identifiable as such in the resulting object?
 #• Does the “concatenation axis” contain data that needs to be preserved? In many cases, the default integer labels in a DataFrame are best discarded during concatenation.

#The concat function in pandas provides a consistent way to address each of these questions.
#I’ll give a number of examples to illustrate how it works.
#Suppose we have three Series with no index overlap:

In [ ]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")

In [ ]:
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")

In [ ]:
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [ ]:
#Calling pandas.concat with these objects in a list glues together the values and indexes

In [ ]:
s1

,0
a,0
b,1


In [ ]:
s2

,0
c,2
d,3
e,4


In [ ]:
s3

,0
f,5
g,6


In [ ]:
pd.concat([s1, s2, s3])

,0
a,0
b,1
c,2
d,3
e,4
f,5
g,6


In [ ]:
#By default, pandas.concat works along axis="index", producing another Series. If
#you pass axis="columns", the result will instead be a DataFrame:

In [ ]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [ ]:
#In this case there is no overlap on the other axis, which as you can see is the union (the "outer" join) of the indexes. You can instead intersect them by passing join="inner":

In [ ]:
s4 = pd.concat([s1, s3])

In [ ]:
s4

,0
a,0
b,1
f,5
g,6


In [ ]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [ ]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [ ]:
#A potential issue is that the concatenated pieces are not identifiable in the result.
#Suppose instead you wanted to create a hierarchical index on the concatenation axis.
#To do this, use the keys argument:

In [ ]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])

In [ ]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [ ]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [ ]:
#In the case of combining Series along axis="columns", the keys become the Data‐Frame column headers:

In [ ]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [ ]:
#The same logic extends to DataFrame objects:

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
 .....: columns=["one", "two"])

In [ ]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
 .....: columns=["three", "four"])

In [ ]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [ ]:
df2

,three,four
a,5,6
c,7,8


In [ ]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [ ]:
#Here the keys argument is used to create a hierarchical index where the first level can
#be used to identify each of the concatenated DataFrame objects.

#If you pass a dictionary of objects instead of a list, the dictionary’s keys will be used
#for the keys option:

In [ ]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [ ]:
#we can name the created axis levels with the names argument

In [ ]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
 .....: names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [ ]:
#A last consideration concerns DataFrames in which the row index does not contain any relevant data:

In [ ]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
 .....: columns=["a", "b", "c", "d"])

In [ ]:
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
 .....: columns=["b", "d", "a"])

In [ ]:
df1

,a,b,c,d
0,0.025554,0.876202,0.077748,1.608728
1,-1.300794,1.228416,0.473108,-0.169700
2,-0.404758,-0.369397,-0.170809,-0.447308


In [ ]:
df2

,b,d,a
0,-1.474342,0.184271,0.551633
1,0.244058,-1.333393,0.404272


In [ ]:
#In this case, you can pass ignore_index=True, which discards the indexes from each
#DataFrame and concatenates the data in the columns only, assigning a new default index:

In [ ]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.025554,0.876202,0.077748,1.608728
1,-1.300794,1.228416,0.473108,-0.169700
2,-0.404758,-0.369397,-0.170809,-0.447308
3,0.551633,-1.474342,NaN,0.184271
4,0.404272,0.244058,NaN,-1.333393


In [ ]:
#Combining Data with Overlap

In [ ]:
#There is another data combination situation that can’t be expressed as either a merge
#or concatenation operation. You may have two datasets with indexes that overlap in
#full or in part. As a motivating example, consider NumPy’s where function, which
#performs the array-oriented equivalent of an if-else expression:

In [ ]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
 index=["f", "e", "d", "c", "b", "a"])

In [ ]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
 index=["a", "b", "c", "d", "e", "f"])

In [ ]:
a

,0
f,NaN
e,2.5
d,0.0
c,3.5
b,4.5
a,NaN


In [ ]:
b

,0
a,0.0
b,NaN
c,2.0
d,NaN
e,NaN
f,5.0


In [ ]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [ ]:
#Here, whenever values in a are null, values from b are selected, otherwise the nonnull values from a are selected. Using numpy.where does not check whether the index
#labels are aligned or not (and does not even require the objects to be the same length), so if you want to line up values by index, use the Series combine_first method:

In [ ]:
a.combine_first(b)

,0
a,0.0
b,4.5
c,3.5
d,0.0
e,2.5
f,5.0


In [ ]:
#With DataFrames, combine_first does the same thing column by column, so you
#can think of it as “patching” missing data in the calling object with data from the
#object you pass:

In [ ]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
 .....: "b": [np.nan, 2., np.nan, 6.],
 .....: "c": range(2, 18, 4)})

In [ ]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
 .....: "b": [np.nan, 3., 4., 6., 8.]})

In [ ]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [ ]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [ ]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [ ]:
#The output of combine_first with DataFrame objects will have the union of all the column names:

In [ ]:
#8.3 Reshaping and Pivoting

In [ ]:
#Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
#There are two primary actions:

#stack
#This “rotates” or pivots from the columns in the data to the rows.

#unstack
#This pivots from the rows into the columns.
#Consider a small DataFrame with string arrays as row and column indexes:

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
 .....: index=pd.Index(["Ohio", "Colorado"], name="state"),
 .....: columns=pd.Index(["one", "two", "three"],
 .....: name="number"))

In [ ]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [ ]:
#Using the stack method on this data pivots the columns into the rows, producing a Series:

In [ ]:
result = data.stack()

In [ ]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [ ]:
#From a hierarchically indexed Series, you can rearrange the data back into a Data‐
#Frame with unstack:

In [ ]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [ ]:
#By default, the innermost level is unstacked (same with stack). You can unstack a
#different level by passing a level number or name:

In [ ]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [ ]:
#Unstacking might introduce missing data if all of the values in the level aren’t found
#in each subgroup:

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")

In [ ]:
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")

In [ ]:
data2 = pd.concat([s1, s2], keys=["one", "two"])

In [ ]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [ ]:
#Stacking filters out missing data by default, so the operation is more easily invertible:

In [ ]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [ ]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [ ]:
data2.unstack().stack(dropna=False)

/tmp/ipython-input-3936770077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [ ]:
#When you unstack in a DataFrame, the level unstacked becomes the lowest level in
#the result:

In [ ]:
df = pd.DataFrame({"left": result, "right": result + 5},
 .....: columns=pd.Index(["left", "right"], name="side"))

In [ ]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [ ]:
#As with unstack, when calling stack we can indicate the name of the axis to stack:

In [ ]:
df.unstack(level="state").stack(level="side")

/tmp/ipython-input-2617337668.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack(level="state").stack(level="side")


state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [ ]:
#Pivoting "Long" to "Wide" Format

In [ ]:
#A common way to store multiple time series in databases and CSV files is what
#is sometimes called long or stacked format. In this format, individual values are
#represented by a single row in a table rather than multiple values per row.
#Let’s load some example data and do a small amount of time series wrangling and
#other data cleaning

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/wesm/pydata-book/a892791ce5a3831134cfa2989033ecbb9289f1ea/examples/macrodata.csv")

In [3]:
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

In [4]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [5]:
#First, I use pandas.PeriodIndex (which represents time intervals rather than points in time), discussed in more detail in Chapter 11, to combine the year and quarter
#columns to set the index to consist of datetime values at the end of each quarter:

In [6]:
periods = pd.PeriodIndex(year=data.pop("year"), quarter=data.pop("quarter"), name="date")

/tmp/ipython-input-34644899.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"), quarter=data.pop("quarter"), name="date")


In [7]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [8]:
data.index = periods.to_timestamp("D")

In [9]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [10]:
#I select a subset of columns and give the columns index the name "item":

In [11]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])

In [12]:
data.columns.name = "item"

In [13]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [14]:
#Lastly, I reshape with stack, turn the new index levels into columns with
#reset_index, and finally give the column containing the data values the name "value":

In [15]:
long_data = data.stack()

In [16]:
long_data[:10]

date        item   
1959-01-01  realgdp    2710.349
            infl          0.000
            unemp         5.800
1959-04-01  realgdp    2778.801
            infl          2.340
            unemp         5.100
1959-07-01  realgdp    2775.488
            infl          2.740
            unemp         5.300
1959-10-01  realgdp    2785.204
dtype: float64

In [17]:
long_data = (data.stack().reset_index().rename(columns={0: "value"}))

In [18]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [ ]:
#In this so-called long format for multiple time series, each row in the table represents a single observation.
#Data is frequently stored this way in relational SQL databases, as a fixed schema (column names and data types) allows the number of distinct values in the item column
#to change as data is added to the table. In the previous example, date and item would
#usually be the primary keys (in relational database parlance), offering both relational
#integrity and easier joins. In some cases, the data may be more difficult to work with
#in this format; you might prefer to have a DataFrame containing one column per
#distinct item value indexed by timestamps in the date column. DataFrame’s pivot
#method performs exactly this transformation:

In [19]:
pivoted = long_data.pivot(index="date", columns="item", values="value")

In [20]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [21]:
#The first two values passed are the columns to be used, respectively, as the row and
#column index, then finally an optional value column to fill the DataFrame. Suppose
#you had two value columns that you wanted to reshape simultaneously:

In [22]:
long_data["value2"] = np.random.standard_normal(len(long_data))

In [23]:
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,1.000881
1,1959-01-01,infl,0.000,0.809128
2,1959-01-01,unemp,5.800,-0.392388
3,1959-04-01,realgdp,2778.801,0.489804
4,1959-04-01,infl,2.340,1.562970
5,1959-04-01,unemp,5.100,1.890064
6,1959-07-01,realgdp,2775.488,-1.228788
7,1959-07-01,infl,2.740,-0.516363
8,1959-07-01,unemp,5.300,0.648772
9,1959-10-01,realgdp,2785.204,0.184966


In [24]:
#By omitting the last argument, you obtain a DataFrame with hierarchical columns:

In [26]:
pivoted = long_data.pivot(index="date", columns="item")

In [27]:
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.809128  1.000881 -0.392388
1959-04-01  2.34  2778.801   5.1  1.562970  0.489804  1.890064
1959-07-01  2.74  2775.488   5.3 -0.516363 -1.228788  0.648772
1959-10-01  0.27  2785.204   5.6 -1.365184  0.184966 -0.922684
1960-01-01  2.31  2847.699   5.2 -0.877799 -0.306092 -0.519467

In [28]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [29]:
#Note that pivot is equivalent to creating a hierarchical index using set_index followed by a call to unstack:

In [30]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")

In [31]:
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.809128  1.000881 -0.392388
1959-04-01  2.34  2778.801   5.1  1.562970  0.489804  1.890064
1959-07-01  2.74  2775.488   5.3 -0.516363 -1.228788  0.648772
1959-10-01  0.27  2785.204   5.6 -1.365184  0.184966 -0.922684
1960-01-01  2.31  2847.699   5.2 -0.877799 -0.306092 -0.519467

In [32]:
#Pivoting "Wode" to "Long" Format

In [33]:
#An inverse operation to pivot for DataFrames is pandas.melt. Rather than trans‐
#forming one column into many in a new DataFrame, it merges multiple columns into
#one, producing a DataFrame that is longer than the input. Let’s look at an example:

In [34]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"], "A": [1, 2, 3], "B": [4, 5, 6], "C": [7, 8, 9]})

In [35]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [36]:
#The "key" column may be a group indicator, and the other columns are data values.
#When using pandas.melt, we must indicate which columns (if any) are group indicators.
#Let’s use "key" as the only group indicator here:

In [37]:
melted = pd.melt(df, id_vars="key")

In [38]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [39]:
#Using pivot, we can reshape back to the original layout:

In [40]:
reshaped = melted.pivot(index="key", columns="variable",
 .....: values="value")

In [41]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [43]:
#Since the result of pivot creates an index from the column used as the row labels, we
#may want to use reset_index to move the data back into a column:

In [44]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [45]:
#You can also specify a subset of columns to use as value columns:

In [46]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [47]:
#pandas.melt can be used without any group identifiers, too:

In [48]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [49]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
